In [1]:
import pandas as pd
from sqlalchemy import create_engine
from elo_calculations import elo_update, process_maps

In [2]:

# Database connection
def get_db_connection():
    user = 'postgres'
    password = 'Ballin23!'
    host = 'localhost'
    port = '5433'
    database = 'postgres'

    connection_string = f'postgresql://{user}:{password}@{host}:{port}/{database}'
    engine = create_engine(connection_string)
    return engine.connect(), engine

In [3]:
def fetch_latest_elos():
    conn, _ = get_db_connection()
    query = 'SELECT * FROM latest_elo_ratings'
    df = pd.read_sql(query, conn)
    conn.close()

    # Create a dictionary of dictionaries to store Elo ratings by map and team
    elo_ratings = {}
    for _, row in df.iterrows():
        team = row['team']
        map_name = row['map']
        rating = row['elo_rating']

        if map_name not in elo_ratings:
            elo_ratings[map_name] = {}
        elo_ratings[map_name][team] = rating

    return elo_ratings

In [4]:
def fetch_new_matches():
    conn, _ = get_db_connection()
    query = '''
    SELECT * FROM valorant_game_data
    WHERE match_id NOT IN (SELECT match_id FROM processed_matches)
    ORDER BY match_timestamp
    '''
    df = pd.read_sql(query, conn)
    conn.close()
    return df

In [5]:
fetch_new_matches()

,match_id,match_timestamp,winning_team,losing_team,winning_team_score,losing_team_score,map,region,event,league
